In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import transformers
import torch
from dotenv import load_dotenv
import os
import openai
import banking  # noqa: E402

load_dotenv(".env")

open_ai_key = os.environ.get("openai-key")


In [2]:
_ = banking.BankingData("https://tinyurl.com/jb-bank", "bank")
_.extract_to_csv()

# Loading in SQL extension
%reload_ext sql
# Initiating a DuckDB database named 'bank.duck.db' to run our SQL queries on
%sql duckdb:///bank.duck.db

In [3]:
%%sql
CREATE OR REPLACE TABLE bank AS
FROM read_csv_auto('bank_cleaned.csv', header=True, sep=',')

Running query in 'duckdb:///bank.duck.db'

Count
4521


In [4]:
columns = %sql PRAGMA table_info('bank');

# Extract column names
column_names = [row[1] for row in columns]

Running query in 'duckdb:///bank.duck.db'

<h1 align='center'>Prompts & Agents</h1>

<h2 align='center'>How to incorporate prompting into your Python scripts <br>and expand their functionality through agents</h2>

<h2 align='center'>Laura Funderburk</h2>

<h2 align='center'>PyData Vancouver</h2>

<h1 align='center'>Talk at a glance</h1>

<h2 align='center'>Part I: Prompting (25 minutes)</h2>


1. LLMs use cases and tasks
2. The Generative AI project lifecycle 
3. Choosing the right LLM for the desired task
4. Key elements of prompting & Prompting techniques
5. Prompting private LLMs (OpenAI API): `ChatCompletion`
6. Prompting open source LLMs through HuggingFace


Q&A about prompting (5 minutes)

<h1 align='center'>Talk at a glance</h1>

<h2 align='center'>Part II: Incorporating agents (25 minutes)</h2>


1. (Some) Applications of agents
2. Introduction to Haystack
3. Introduction to LangChain
4. Which one to pick
5. Techniques to combine prompting and agents

Q&A about agents (5 minutes)

<h1 align='center'>LLMs use cases and tasks</h1>


<h1 align='center'>LLMs use cases and tasks</h1>


* Text summarization

* Conversation 

* Translation

* Text generation

* Text, token and sentiment classification

* Table Q&A and Q&A from unstructured data

* Sentence similarity

* Masking

<h1 align='center'>LLMs use cases and tasks</h1>

<h2 align='center'>Your goal is to understand the business case you are solving -<br> then select the appropriate methods to solve it</h2>

<h3 align='center'>Who will benefit from your product?</h3>

<h3 align='center'>What is the end result?</h3>

<h3 align='center'>How will it be served?</h3>


<h1 align='center'>The generative AI project lifecycle</h1>


<h1 align='center'>The generative AI project lifecycle</h1>

<p></p>

<center>
  <img src="diagrams/genai_project_lifecycle.jpg" width="1400px"/>

</center>

<h1 align='center'>Focus of this talk</h1>

<p></p>
<center>
  <img src="diagrams/genai_project_lifecycle_focus.jpg" width="1400px"/>

</center>

<h1 align='center'>Choosing the right LLM (architecture) for the desired task</h1>

<p></p>
<center>
  <img src="diagrams/opt.jpeg" width="200px"/>

</center>

* GPT-like (also called auto-regressive Transformer models)
* BERT-like (also called auto-encoding Transformer models)
* BART/T5-like (also called sequence-to-sequence Transformer models)



<h1 align='center'>Choosing the right LLM (architecture) for the desired task</h1>

* Encoder-only models: Good for tasks that require **understanding of the input** 
* Decoder-only models: Good for **generative tasks** 
* Encoder-decoder models or sequence-to-sequence models: Good for **generative tasks that require an input** 


<h1 align='center'>Do I need to train a new model to solve my problem?</h1>

<h3 align='center'>No. Training a LLM is costly (GPU usage, time, resources, data). This is why sharing LLMs and their fine-tuned componets has become highly popularized.</h3>


<p></p>
<center>
  <img src="diagrams/hftasks.png" width="1300px"/>

</center>

Source: https://huggingface.co/tasks

<h1 align='center'>Key elements of prompting</h1>

<h3 align='center'>Basic</h3>

* A LLM to interact with
* Temperature
* Max tokens
* A natural language request

<h3 align='center'>Advanced</h3>

* Data (text files, web files)
* A database storage system (vector DB, SQL, PostgreSQL, etc)


<h1 align='center'>Prompting techniques</h1>

* Zero-shot inference

* One-shot inference

* Few-shot inference


<h1 align='center'>Prompting techniques: Zero-shot inference</h1>

Formula: prompt, no examples.

Suppose we want to translate natural language queries into SQL. 

We have a `natural_question`, a database table name `db_name` and a schema `schema`

```python
prompt= f"Answer the question {natural_question} for table {db_name} with schema {schema}"        
```

Suppose we want to classify the sentiment in a sentence `sentence`

```python
prompt = f"How does the author feel about this based on the statement {sentence}"

```

<h1 align='center'>Prompting techniques: One-shot inference</h1>

Formula: prompt, one example.

```python
prompt= f"Answer the question {natural_question} for table {db_name} with schema {schema}\
                        For example, if you receive the question: 'How many records are there?'\
                        An appropriate answer is\
                        'SELECT COUNT(*) FROM bank'"
```

```python
prompt = f"How does the author feel about this based on the statement {sentence}\
            'I find the user experience is confusing and convoluted'\
            Answer: Negative"

```

<h1 align='center'>Prompting techniques: Few-shot inference</h1>

More than one example.

```python
prompt= f"Answer the question {natural_question} for table {db_name} with schema {schema}\
                        For example, if you receive the question: 'How many records are there?'\
                        An appropriate answer is\
                        'SELECT COUNT(*) FROM bank'"
```

```python
prompt = f"How does the author feel about this based on the statement {sentence}\
            'I find the user experience is confusing and convoluted'\
            Answer: Negative\
            'The decoration of the room made me feel welcome!'\
            Answer: Positive"

```

<h1 align='center'>Prompting private LLMs (OpenAI API)</h1>


We're going to focus on the `ChatCompletion` end point. 

Key elements:

* OpenAI API Key
* Model chosen (GPT4, GPT 3.5 Turbo, Text-Davinci)
* Temperature


<h1 align='center'>Prompting private LLMs (OpenAI API)</h1>


We're going to focus on the `ChatCompletion` end point. 

```python
class Prompter:
    def __init__(self, api_key, gpt_model, temperature=0.2):
        if not api_key:
            raise Exception("Please provide the OpenAI API key")

        self.api_key  = api_key
        self.gpt_model = gpt_model
        self.temperature = temperature
    
    def prompt_model_return(self, messages: list):
        openai.api_key = self.api_key
        response = openai.ChatCompletion.create(model=self.gpt_model, 
                                                messages=messages,
                                                temperature=self.temperature)
        return response["choices"][0]["message"]["content"]
    
```

<h1 align='center'>Roles in prompting the ChatCompletion endpoint</h1>

`system content`: What context should the LLM have in mind? Expert in marketing, helpful assistant, enthusiastic marketing generator.

`user content`: What are typical requests that someone with that role would receive?


<h1 align='center'>Roles in prompting the ChatCompletion endpoint</h1>


We're going to focus on the `ChatCompletion` end point. 

```python
    def natural_language_to_sql(self, db_name:str, schema:str, natural_question:str):

            system_content = f"You are a data analyst, \
                               and you specialize in solving business questions with SQL.\
                               You are given a natural language question, \
                               and your role is to translate the question\
                               into a query that can be executed against a database. \
                               Ensure your queries are written in a single line, with no special characters"
            
            user_content = f"Please generate a SQL query for data with in a database named {db_name}\
                            along with a schema {schema} for the question {natural_question}"

            full_prompts = [
                                    {"role" : "system", "content" : system_content},
                                    {"role" : "user", "content" : user_content},
                                    ]

            result = self.prompt_model_return(full_prompts)

            return result
    
```

In [5]:
class Prompter:
    def __init__(self, api_key, gpt_model, temperature=0.2):
        if not api_key:
            raise Exception("Please provide the OpenAI API key")

        self.api_key  = api_key
        self.gpt_model = gpt_model
        self.temperature = temperature
    
    def chat_completion(self, messages: list):
        openai.api_key = self.api_key
        response = openai.ChatCompletion.create(model=self.gpt_model, 
                                                messages=messages,
                                                temperature=self.temperature)
        return response["choices"][0]["message"]["content"]
    
    def natural_language_to_sql(self, db_name:str, schema:str, natural_question:str):

        system_content = f"You are a data analyst, and you specialize in solving business questions with SQL.\
                        You are given a natural language question, and your role is to translate the question\
                        into a query that can be executed against a database. \
                        Ensure your queries are written in a single line, with no special characters"
        user_content = f"Please generate a SQL query for data with in a database named {db_name}\
                        along with a schema {schema} for the question {natural_question}"

        full_prompts = [
                                {"role" : "system", "content" : system_content},
                                {"role" : "user", "content" : user_content},
                                ]
        
        result = self.chat_completion(full_prompts)

        return result
   
    
    def natural_language_zero_shot(self, db_name:str, schema:str, natural_question:str):
        user_content= f"Answer the question {natural_question} for table {db_name} with schema {schema}"
        
        full_prompt = [{"role" : "user", "content" : user_content}]
        
        
        result = self.chat_completion(full_prompt)

        return result
    
    def natural_language_single_shot(self, db_name:str, schema:str, natural_question:str):
        user_content= f"Answer the question {natural_question} for table {db_name} with schema {schema}\
                        For example, if you receive the question: 'How many records are there?'\
                        An appropriate answer is\
                        'SELECT COUNT(*) FROM bank'"
        
        full_prompt = [{"role" : "user", "content" : user_content}]
        
        
        result = self.chat_completion(full_prompt)

        return result
    
    def natural_language_few_shot(self, db_name:str, schema:str, natural_question:str):
        user_content= f"Answer the question {natural_question} for table {db_name} with schema {schema}\
                        For example, if you receive the question: 'How many records are there?'\
                        'SELECT COUNT(*) FROM bank'\
                        If you receive the question: Find all employees that are unemployed\
                        SELECT * FROM bank WHERE job = 'unemployed'"
        
        full_prompt = [{"role" : "user", "content" : user_content}]
        
        
        result = self.chat_completion(full_prompt)

        return result
    

In [6]:
pm  = Prompter(open_ai_key, "gpt-3.5-turbo")

pm.natural_language_to_sql("bank", column_names, "How many unique jobs are there?")

'SELECT COUNT(DISTINCT job) FROM bank'

In [ ]:
pm.natural_language_zero_shot("bank", column_names, "How many unique jobs are there?")

In [ ]:
pm.natural_language_single_shot("bank", column_names, "How many unique jobs are there?")

<h1 align='center'>Prompting open source LLMs through HuggingFace</h1>


You only need to ensure you install the right modules via `pip` along with the model card of the LLM.

```
<user_name>/<model_name>
```
<p></p>
<center>
  <img src="diagrams/tap.png" width="800px"/>

</center>

Source: https://huggingface.co/microsoft/tapex-base

<h1 align='center'>Prompting open source LLMs through HuggingFace</h1>


You only need to ensure you install the right modules via `pip` along with the model card of the LLM.

```
<user_name>/<model_name>
```

When you execute the download code you should see something like

<p></p>
<center>
  <img src="diagrams/model_download.png" width="800px"/>

</center>



In [ ]:
from transformers import TapexTokenizer, BartForConditionalGeneration
import pandas as pd


class OpenSourcePrompter:
    def __init__(self, model_name):
        self.model_name = model_name
        
    def call_model(self, full_prompt):
        model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_fast=True)
        
        inputs = tokenizer(full_prompt, return_tensors='pt')
        output = tokenizer.decode(
            model.generate(
                inputs["input_ids"], 
                max_new_tokens=50,
            )[0], 
            skip_special_tokens=True
        )
        
        return output
        
    def natural_language_to_sql(self, db_name:str, schema:list, natural_question:str):

        prompt = f"Given the natural language question {natural_question}\
                        the database table {db_name}, \
                        and the table schema {', '.join(schema)}\
                        generate a SQL query that answers the question {natural_question}"

    
        result = self.call_model(prompt)

        return result

In [ ]:
from transformers import TapexTokenizer, BartForConditionalGeneration
import pandas as pd


class TapexTokenizer:
    def __init__(self, table):
        self.table = table
        
    def tapex_tokenizer(self, query):
        
        tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-base")
        model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base")
            
        # tapex accepts uncased input since it is pre-trained on the uncased corpus
        query = "select year where city = beijing"
        encoding = tokenizer(table=self.table, query=query, return_tensors="pt")

        outputs = model.generate(**encoding)

        return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:

model_name='google/flan-t5-base'
opm = OpenSourcePrompter(model_name)


In [ ]:
opm.natural_language_to_sql("bank", column_names, "How many records are there?")

In [ ]:
%sqlcmd explore --table bank

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
from haystack.nodes import PromptNode

# Initalize the node passing the model:
prompt_node = PromptNode(model_name_or_path="google/flan-t5-base")


In [ ]:
# Go ahead and ask a question:
prompt_node("What is the best city in Europe to live in?")


In [ ]:
{"system"}

In [ ]:
pn = PromptNode("gpt-4", api_key=open_ai_key)


In [ ]:
prompt = "What is the best city in Europe to live in?"
pn(prompt, max_tokens=100)